## Creating components with pyOpTools

To be able to simulate an optical system, the first step is to use the predefined surfaces to create components. In this notebook it will be shown how to create some simple components.

In [ ]:
from pyoptools.all import *
from numpy import pi, sqrt

### Creating a bi-convex lens

A bi-convex lens can described as a piece of glass limited by 2 spherical surfaces (the 2 lens faces), and a a cylindrical surface (the lens border). In the next example, this surfaces will be:

1. An spherical surface limited by a circular shape with 100 mm diameter (this will be the lens diameter), and a curvature radious of 200 mm.
1. A second spherical surface with the same characteristics as the previous.
1. A 100. mm diameter cylinder with 10 mm length.

<div class="alert alert-info">
    
**Note:** In pyOpTools the dimensions are always given in millimeters.

</div>

In [ ]:
S0 = Spherical(shape=Circular(radius=50), curvature=1.0 / 200.0)
S1 = Spherical(shape=Circular(radius=50), curvature=1.0 / 200.0)
S2 = Cylinder(radius=50, length=10)

With the created surfaces we will create the Component. The component constructor receives 2 important arguments:

**surflist**

> List of tuples of the form `(surface, (PosX,PosY,PosZ), (RotX,RotY,RotZ)` where `surface` is an instance of a subclass of [Surface](../../pyoptools.raytrace.surface.surface.rst), `PosX,PosY,PosZ`are the surface’s vertex coordinates, and `RotX,RotY,RotZ` are the rotation angles of the surface around the X , Y , and Z axes, given in radians. The rotation about the Z axis if applied first, then the rotation about the Y axis, and finally the rotation about the X axis.

**material**

> Instance of the class Material with the material definition, or a floating point number to indicate a constant refraction index material.


In [ ]:
L1 = Component(
    surflist=[
        (S0, (0, 0, -5), (0, 0, 0)),
        (S1, (0, 0, 5), (0, 0, 0)),
        (S2, (0, 0, 6.5), (0, 0, 0)),
    ],
    material=material.schott["N-BK7"],
)
Plot3D(L1, size=(120, 120), scale=3, rot=[(pi / 3, 0, 0)])

### Creting a 90 degree prism

The next code is an example of the creation of a right angle prism. This component is composed as 3 rectangular planes and two triangular planes.

In [ ]:
width = 50
height = 50
reflectivity = 0.5

a_face = Plane(shape=Rectangular(size=(width, height)))
b_face = Plane(shape=Rectangular(size=(width, height)))


h = sqrt(2.0) * width
h_face = Plane(shape=Rectangular(size=(h, height)), reflectivity=reflectivity)

w2 = width / 2.0

e1 = Plane(shape=Triangular(((-w2, w2), (-w2, -w2), (w2, -w2))))
e2 = Plane(shape=Triangular(((-w2, w2), (-w2, -w2), (w2, -w2))))

In [ ]:
P = Component(
    surflist=[
        (a_face, (0, 0, -width / 2), (0, 0, 0)),
        (b_face, (width / 2, 0, 0), (0, pi / 2, 0)),
        (h_face, (0, 0, 0), (0, -pi / 4, 0)),
        (e1, (0, height / 2, 0), (pi / 2, -pi / 2, 0)),
        (e2, (0, -height / 2, 0), (pi / 2, -pi / 2, 0)),
    ],
    material=material.schott["N-BK7"],
)
Plot3D(P, size=(120, 120), scale=3, rot=[(pi / 6, pi / 8, 0)])

### Creating a beam splitting cube

The next code shows an example to create a beam splitting cube by using 2 components (prisms defined by a function) to create a system (in this case it is really a subsystem that can be used as a component). An extra feature added to this example, is a _reflective_ characteristic added to one of the surfaces in the prism `P2`, so the subsystem behaves as a beam splitting cube.

<div class="alert alert-warning">
    
**Warning:**

1. Care must be taken when using reflective surfaces to avoid the creation of resonnant cavities such as 2 parallel semi-reflective surfaces, as pyOpTools will try to propagate the rays for ever and the system will crash. For this reason, when creating the beam splitting cube, only one of the prisms have a reflective surface.

2. pyOpTools can handle 2 surfaces (from different components) in contact, but it can not detect nor handle if the components overlap in space, and will provide an erroneous result. 

</div>
 

In [ ]:
def prism(reflectivity=0):
    width = 50
    height = 50
    a_face = Plane(shape=Rectangular(size=(width, height)))
    b_face = Plane(shape=Rectangular(size=(width, height)))

    h = sqrt(2.0) * width
    h_face = Plane(shape=Rectangular(size=(h, height)), reflectivity=reflectivity)

    w2 = width / 2.0

    e1 = Plane(shape=Triangular(((-w2, w2), (-w2, -w2), (w2, -w2))))
    e2 = Plane(shape=Triangular(((-w2, w2), (-w2, -w2), (w2, -w2))))
    P = Component(
        surflist=[
            (a_face, (0, 0, -width / 2), (0, 0, 0)),
            (b_face, (width / 2, 0, 0), (0, pi / 2, 0)),
            (h_face, (0, 0, 0), (0, -pi / 4, 0)),
            (e1, (0, height / 2, 0), (pi / 2, -pi / 2, 0)),
            (e2, (0, -height / 2, 0), (pi / 2, -pi / 2, 0)),
        ],
        material=material.schott["N-BK7"],
    )
    return P


P1 = prism()
P2 = prism(reflectivity=0.5)

cube = System(complist=[(P1, (0, 0, 0), (0, 0, 0)), (P2, (0, 0, 0), (0, pi, 0))], n=1.0)

Plot3D(cube, size=(120, 120), scale=3, rot=[(pi / 6, pi / 8, 0)])